# minimal example of ontram implementation
- anlogous to https://github.com/liherz/ontram_pytorch.git

In [1]:
# Load dependencies
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split



from utils.configuration import *
from utils.loss_ordinal import *
from utils.tram_model_helpers import *
from utils.tram_models import *
from utils.tram_data import *


import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Train with GPU support.")
else:
    device = torch.device('cpu')
    print("No GPU found, train with CPU support.")

Train with GPU support.


adjustet funcitnos for ordinal outcomes 

dev ordinal

In [2]:
experiment_name = "ordinal_wine_example_7"   ## <--- set experiment name
seed=42
np.random.seed(seed)

LOG_DIR="/home/bule/TramDag/dev_experiment_logs"
EXPERIMENT_DIR = os.path.join(LOG_DIR, experiment_name)
DATA_PATH = EXPERIMENT_DIR # <----------- change to different source if needed
CONF_DICT_PATH = os.path.join(EXPERIMENT_DIR, f"configuration.json")

In [3]:
# Load the dataset
wine = load_wine()
wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [4]:
df=pd.DataFrame(wine['data'], columns=wine['feature_names'])
df['target']=wine['target']

In [5]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

quantiles = train_df.quantile([0.05, 0.95])
min_vals = quantiles.loc[0.05]
max_vals = quantiles.loc[0.95]

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 142 entries, 158 to 102
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       142 non-null    float64
 1   malic_acid                    142 non-null    float64
 2   ash                           142 non-null    float64
 3   alcalinity_of_ash             142 non-null    float64
 4   magnesium                     142 non-null    float64
 5   total_phenols                 142 non-null    float64
 6   flavanoids                    142 non-null    float64
 7   nonflavanoid_phenols          142 non-null    float64
 8   proanthocyanins               142 non-null    float64
 9   color_intensity               142 non-null    float64
 10  hue                           142 non-null    float64
 11  od280/od315_of_diluted_wines  142 non-null    float64
 12  proline                       142 non-null    float64
 13  target  

In [7]:
train_df['target']

158    2
137    2
98     1
159    2
38     0
      ..
71     1
106    1
14     0
92     1
102    1
Name: target, Length: 142, dtype: int64

In [8]:
train_df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
158,14.34,1.68,2.70,25.0,98.0,2.80,1.31,0.53,2.70,13.00,0.57,1.96,660.0,2
137,12.53,5.51,2.64,25.0,96.0,1.79,0.60,0.63,1.10,5.00,0.82,1.69,515.0,2
98,12.37,1.07,2.10,18.5,88.0,3.52,3.75,0.24,1.95,4.50,1.04,2.77,660.0,1
159,13.48,1.67,2.64,22.5,89.0,2.60,1.10,0.52,2.29,11.75,0.57,1.78,620.0,2
38,13.07,1.50,2.10,15.5,98.0,2.40,2.64,0.28,1.37,3.70,1.18,2.69,1020.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,13.86,1.51,2.67,25.0,86.0,2.95,2.86,0.21,1.87,3.38,1.36,3.16,410.0,1
106,12.25,1.73,2.12,19.0,80.0,1.65,2.03,0.37,1.63,3.40,1.00,3.17,510.0,1
14,14.38,1.87,2.38,12.0,102.0,3.30,3.64,0.29,2.96,7.50,1.20,3.00,1547.0,0
92,12.69,1.53,2.26,20.7,80.0,1.38,1.46,0.58,1.62,3.05,0.96,2.06,495.0,1


data types

In [9]:
data_type={key:value for key, value in zip(train_df.columns, ['continous']*13+['ordinal_Xc_Yo'])}
data_type

{'alcohol': 'continous',
 'malic_acid': 'continous',
 'ash': 'continous',
 'alcalinity_of_ash': 'continous',
 'magnesium': 'continous',
 'total_phenols': 'continous',
 'flavanoids': 'continous',
 'nonflavanoid_phenols': 'continous',
 'proanthocyanins': 'continous',
 'color_intensity': 'continous',
 'hue': 'continous',
 'od280/od315_of_diluted_wines': 'continous',
 'proline': 'continous',
 'target': 'ordinal_Xc_Yo'}

configartion dicitonary

In [10]:
configuration_dict=new_conf_dict(experiment_name,EXPERIMENT_DIR,DATA_PATH,LOG_DIR)
configuration_dict

{'date_of_creation': '2025-07-31 14:50:52',
 'experiment_name': 'ordinal_wine_example_7',
 'PATHS': {'DATA_PATH': '/home/bule/TramDag/dev_experiment_logs/ordinal_wine_example_7',
  'LOG_DIR': '/home/bule/TramDag/dev_experiment_logs',
  'EXPERIMENT_DIR': '/home/bule/TramDag/dev_experiment_logs/ordinal_wine_example_7'},
 'data_type': None,
 'adj_matrix': None,
 'model_names': None,
 'seed': None,
 'nodes': None}

modeloling it as a graph with one sink node 

In [11]:
data_type={key:value for key, value in zip(train_df.columns, ['continous']*13+['ordinal_Xc_Yo'])}


columns = [
    'alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
    'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins',
    'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline', 'target'
]

adj_matrix = np.full((len(columns), len(columns)), "0", dtype=object)

# Set last column (edges *to* 'target') as "ls", excluding self-loop
for i in range(len(columns) - 1):
    adj_matrix[i, -1] = "ls"
    
nn_names_matrix= create_nn_model_names(adj_matrix,data_type)
nn_names_matrix

array([['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        'LinearShift'],
       ['0', '0', '0', '0', '0

In [12]:
levels_dict=create_levels_dict(df,data_type)
target_nodes=create_node_dict(adj_matrix, nn_names_matrix, data_type, min_vals, max_vals,levels_dict)
target_nodes

{'alcohol': {'Modelnr': 0,
  'data_type': 'continous',
  'node_type': 'source',
  'parents': [],
  'parents_datatype': {},
  'transformation_terms_in_h()': {},
  'min': 11.665000000000001,
  'max': 14.2295,
  'transformation_term_nn_models_in_h()': {}},
 'malic_acid': {'Modelnr': 1,
  'data_type': 'continous',
  'node_type': 'source',
  'parents': [],
  'parents_datatype': {},
  'transformation_terms_in_h()': {},
  'min': 1.0710000000000002,
  'max': 4.600999999999998,
  'transformation_term_nn_models_in_h()': {}},
 'ash': {'Modelnr': 2,
  'data_type': 'continous',
  'node_type': 'source',
  'parents': [],
  'parents_datatype': {},
  'transformation_terms_in_h()': {},
  'min': 1.92,
  'max': 2.7495,
  'transformation_term_nn_models_in_h()': {}},
 'alcalinity_of_ash': {'Modelnr': 3,
  'data_type': 'continous',
  'node_type': 'source',
  'parents': [],
  'parents_datatype': {},
  'transformation_terms_in_h()': {},
  'min': 14.030000000000001,
  'max': 25.0,
  'transformation_term_nn_mode

In [13]:
node='target'
tram_model=get_fully_specified_tram_model(node, target_nodes, verbose=True)
tram_model

TramModel(
  (nn_int): SimpleIntercept(
    (fc): Linear(in_features=1, out_features=2, bias=False)
  )
  (nn_shift): ModuleList(
    (0-12): 13 x LinearShift(
      (fc): Linear(in_features=1, out_features=1, bias=False)
    )
  )
)

In [14]:
# from utils.tram_data import get_dataloader

learning_rate=0.001
use_scheduler=False


NODE_DIR = os.path.join(EXPERIMENT_DIR, f'{node}')
os.makedirs(NODE_DIR, exist_ok=True)

MODEL_PATH,LAST_MODEL_PATH,TRAIN_HIST_PATH,VAL_HIST_PATH=model_train_val_paths(NODE_DIR)


optimizer =torch.optim.Adam(tram_model.parameters(), lr=learning_rate)

if use_scheduler:
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
else:
    scheduler = None

train_loader, val_loader = get_dataloader(node, target_nodes, train_df, val_df, batch_size=56,return_intercept_shift=True, verbose=False)


In [ ]:
# TODO initialize with correct weights
epochs =1000
train_val_loop(
            node,
            target_nodes,
            NODE_DIR,
            tram_model,
            train_loader,
            val_loader,
            epochs,
            optimizer,
            use_scheduler,
            scheduler,
            save_linear_shifts=False,
            verbose=1,
            device=device,
            debug=True) # TODO startvalues for bernsteinpols



[DEBUG] Device: cuda
[DEBUG] Model paths: /home/bule/TramDag/dev_experiment_logs/ordinal_wine_example_7/target/best_model.pt /home/bule/TramDag/dev_experiment_logs/ordinal_wine_example_7/target/last_model.pt
[DEBUG] min_max shape: torch.Size([2])
No existing model found. Starting fresh...


Saved new best model.
Epoch 1/1000  Train Loss: 25.7890  Val Loss: 22.5142  [Train: 0.56s  Val: 0.16s  Total: 0.72s]
Epoch 2/1000  Train Loss: 25.7890  Val Loss: 22.5142  [Train: 0.20s  Val: 0.16s  Total: 0.36s]
Epoch 3/1000  Train Loss: 26.3591  Val Loss: 22.5142  [Train: 0.21s  Val: 0.16s  Total: 0.38s]
Epoch 4/1000  Train Loss: 26.5492  Val Loss: 22.5142  [Train: 0.22s  Val: 0.17s  Total: 0.39s]
Epoch 5/1000  Train Loss: 26.5492  Val Loss: 22.5142  [Train: 0.21s  Val: 0.16s  Total: 0.37s]
Epoch 6/1000  Train Loss: 26.3591  Val Loss: 22.5142  [Train: 0.20s  Val: 0.18s  Total: 0.39s]
Epoch 7/1000  Train Loss: 26.9293  Val Loss: 22.5142  [Train: 0.19s  Val: 0.16s  Total: 0.36s]
Epoch 8/1000  Train Loss: 26.3591  Val Loss: 22.5142  [Train: 0.22s  Val: 0.20s  Total: 0.43s]
Epoch 9/1000  Train Loss: 26.1691  Val Loss: 22.5142  [Train: 0.22s  Val: 0.21s  Total: 0.44s]
Epoch 10/1000  Train Loss: 26.5492  Val Loss: 22.5142  [Train: 0.22s  Val: 0.16s  Total: 0.38s]
Epoch 11/1000  Train Loss: 

Exception ignored in: <function _releaseLock at 0x767eb5926310>
Traceback (most recent call last):
  File "/home/bule/anaconda3/envs/tramdag/lib/python3.9/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


Epoch 360/1000  Train Loss: 26.3591  Val Loss: 22.5142  [Train: 0.20s  Val: 0.15s  Total: 0.36s]
Epoch 361/1000  Train Loss: 26.3591  Val Loss: 22.5142  [Train: 0.24s  Val: 0.21s  Total: 0.46s]
Epoch 362/1000  Train Loss: 25.7890  Val Loss: 22.5142  [Train: 0.21s  Val: 0.16s  Total: 0.37s]
Epoch 363/1000  Train Loss: 27.1193  Val Loss: 22.5142  [Train: 0.23s  Val: 0.19s  Total: 0.43s]
Epoch 364/1000  Train Loss: 26.1691  Val Loss: 22.5142  [Train: 0.25s  Val: 0.20s  Total: 0.45s]
Epoch 365/1000  Train Loss: 26.1691  Val Loss: 22.5142  [Train: 0.22s  Val: 0.19s  Total: 0.41s]
Epoch 366/1000  Train Loss: 26.3591  Val Loss: 22.5142  [Train: 0.23s  Val: 0.21s  Total: 0.45s]
Epoch 367/1000  Train Loss: 26.1691  Val Loss: 22.5142  [Train: 0.24s  Val: 0.20s  Total: 0.45s]
Epoch 368/1000  Train Loss: 27.1193  Val Loss: 22.5142  [Train: 0.22s  Val: 0.20s  Total: 0.43s]
Epoch 369/1000  Train Loss: 26.3591  Val Loss: 22.5142  [Train: 0.22s  Val: 0.20s  Total: 0.42s]
Epoch 370/1000  Train Loss: 26

: 

: 

# Evaluate

In [ ]:
## laoding the best model 
MODEL_PATH,LAST_MODEL_PATH,TRAIN_HIST_PATH,VAL_HIST_PATH=model_train_val_paths(NODE_DIR)

if os.path.exists(MODEL_PATH) and os.path.exists(TRAIN_HIST_PATH) and os.path.exists(VAL_HIST_PATH):
    print("Existing model found. Loading weights and history...")
    tram_model.load_state_dict(torch.load(MODEL_PATH))

Existing model found. Loading weights and history...


In [ ]:
## evaluate on testdata
tram_model.eval()

_, ordered_transformation_terms_in_h, _=ordered_parents(node, target_nodes)

min_vals = torch.tensor(target_nodes[node]['min'], dtype=torch.float32).to(device)
max_vals = torch.tensor(target_nodes[node]['max'], dtype=torch.float32).to(device)
min_max = torch.stack([min_vals, max_vals], dim=0)


with torch.no_grad():
    for (int_input, shift_list), y in train_loader:
        int_input = int_input.to(device)
        shift_list = [s.to(device) for s in shift_list]
        y = y.to(device)

        y_pred = tram_model(int_input=int_input, shift_input=shift_list)
        # loss = contram_nll(y_pred, y, min_max=min_max)
        pred_labels = get_pdf_ordinal(get_cdf_ordinal(y_pred)).argmax(dim=1)
        true_labels = y.argmax(dim=1)
        accuracy = (pred_labels == true_labels).float().mean().item()

        print(f"Accuracy: {accuracy*100:.1f}%")  # → 100.0%




with torch.no_grad():
    for (int_input, shift_list), y in val_loader:
        int_input = int_input.to(device)
        shift_list = [s.to(device) for s in shift_list]
        y = y.to(device)

        y_pred = tram_model(int_input=int_input, shift_input=shift_list)
        # loss = contram_nll(y_pred, y, min_max=min_max)
        pred_labels = get_pdf_ordinal(get_cdf_ordinal(y_pred)).argmax(dim=1)
        true_labels = y.argmax(dim=1)
        accuracy = (pred_labels == true_labels).float().mean().item()

        print(f"Accuracy: {accuracy*100:.1f}%")  # → 100.0%

Accuracy: 28.6%
Accuracy: 35.7%
Accuracy: 30.0%
Accuracy: 38.9%
